# importing labraries

In [47]:
import numpy as np
import pandas as pd
import sklearn
import nltk as nlt
import re
import wikipedia
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import LdaModel
import gensim #for text data processing
#from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.gensim_models

# Récupère le contenu de l'article "Python (langage)"

In [48]:
page = wikipedia.page("Python (langage)")
article = page.content

 Afficher le contenu de l'article

In [49]:
print(article)

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation via the off-side rule.Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.Python consistently ranks as one of the most popular programming languages.


== History ==

Python was conceived in the late 1980s by Guido van Rossum at Centrum Wiskunde & Infor

In [50]:
print(len(article))

38153


In [51]:
#Convert the article to a DataFrame object
df = pd.DataFrame([article],columns=["text"])
df.to_csv(r'C:\Users\HP\Desktop\M1\data science\article.csv',index=False,header=True)
data=df.to_csv()

In [52]:
print(data)

,text
0,"Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation via the off-side rule.Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a ""batteries included"" language due to its comprehensive standard library.Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.Python consistently ranks as one of the most popular programming languages.


== History ==

Python was conceived in the late 1980s by Guido van Rossum at Centrum Wisku

In [53]:
df

,text
0,"Python is a high-level, general-purpose progra..."


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1 non-null      object
dtypes: object(1)
memory usage: 136.0+ bytes


# Data cleaning

Text Preprocessing Function (Python)

In [55]:
def preprocess_text(article):
    
    """"
    Cette fonction applique une série de normalisations et de traitements sur un texte donné.
    """
    
    # Normalisation 1 : exemple:ThisIsATest --> This. Is. A. Test(manque de délimiteur)
    article = re.sub(r'([a-z])([A-Z])', r'\1\. \2', article)
    # Normalisation 2 : passage en minuscules
    article = article.lower()
    # Normalisation 3 : répétitions de lettres (si plus de 2)
    # exemple :J'adoreeeeee les frites !" --> J'adore les frites !
    article = re.sub(r'([a-z])\1{2,}', r'\1', article)
    # Suppression de la ponctuation et des chiffres
    processed_article = re.sub(r'[^\w\s]', '', article)
    # Normalisation 4 : contenu entre parenthèses, supposé moins informel
    article = re.sub(r'\(.*?\)', ' ', article)
    # Normalisation 5 : répétition de phrase
    article = re.sub(r'(.{2,}?)\1{1,}', r'\1', article)
    #La méthode strip() est utilisée ici pour supprimer les espaces en début et en fin de chaîne de caractères (au cas où il y en aurait après toutes les normalisations).
    return article.strip()

    # Prétraitement de l'article
processed_article = preprocess_text(article)

In [56]:
print(len(processed_article))

35402


In [57]:
# Displaying preprocessed article
print(processed_article)

python is a high-level, general-purpose programming language. its design philosophy emphasizes code readability with the use of significant indentation via the off-side rule.python is dynamically typed and garbage-collected. it supports multiple programming paradigms, including structured  , object-oriented and functional programming. it is often described as a "batteries included" language due to its comprehensive standard library.guido van rossum began working on python in the late 1980s a successor to the abc programming language and first released it in 1991 as python 0.9.0. python 2.0 was released in 2000. python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. python 2.7.18, released in 20, was the last release of python 2.python consistently ranks as one of the most popular programming languages.


== history ==

python was conceived in the late 1980s by guido van rossum at centrum wiskunde & informatica   in the netherlands a


# Other processing steps

Load the cleaned dataset and apply processing steps. Steps in processing the article:

Tokenization 

Removing the stopwords 

Lemmatization 

In [58]:
#Download the necessary data if they have not already been downloaded
nlt.download('punkt')
# Use the word_tokenize function to tokenize the text
tokens = nlt.word_tokenize(processed_article)
tokens_alphabetic_only=[word for word in tokens if word.isalpha()]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
print(tokens_alphabetic_only)

['python', 'is', 'a', 'programming', 'language', 'its', 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'with', 'the', 'use', 'of', 'significant', 'indentation', 'via', 'the', 'is', 'dynamically', 'typed', 'and', 'it', 'supports', 'multiple', 'programming', 'paradigms', 'including', 'structured', 'and', 'functional', 'programming', 'it', 'is', 'often', 'described', 'as', 'a', 'batteries', 'included', 'language', 'due', 'to', 'its', 'comprehensive', 'standard', 'van', 'rossum', 'began', 'working', 'on', 'python', 'in', 'the', 'late', 'a', 'successor', 'to', 'the', 'abc', 'programming', 'language', 'and', 'first', 'released', 'it', 'in', 'as', 'python', 'python', 'was', 'released', 'in', 'python', 'released', 'in', 'was', 'a', 'major', 'revision', 'not', 'completely', 'with', 'earlier', 'versions', 'python', 'released', 'in', 'was', 'the', 'last', 'release', 'of', 'python', 'consistently', 'ranks', 'as', 'one', 'of', 'the', 'most', 'popular', 'programming', 'languages', 'his

                                                                 Removing the stopwords 
Stopword removal is a common NLP pre-processing step to remove commonly occurring words that do not carry significant meaning in the text.

In [60]:
nlt.download('stopwords')

stop_words = set(stopwords.words('english'))
tokens = [word for word in tokens_alphabetic_only if word.lower() not in stop_words]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [61]:
print(tokens)

['python', 'programming', 'language', 'design', 'philosophy', 'emphasizes', 'code', 'readability', 'use', 'significant', 'indentation', 'via', 'dynamically', 'typed', 'supports', 'multiple', 'programming', 'paradigms', 'including', 'structured', 'functional', 'programming', 'often', 'described', 'batteries', 'included', 'language', 'due', 'comprehensive', 'standard', 'van', 'rossum', 'began', 'working', 'python', 'late', 'successor', 'abc', 'programming', 'language', 'first', 'released', 'python', 'python', 'released', 'python', 'released', 'major', 'revision', 'completely', 'earlier', 'versions', 'python', 'released', 'last', 'release', 'python', 'consistently', 'ranks', 'one', 'popular', 'programming', 'languages', 'history', 'python', 'conceived', 'late', 'guido', 'van', 'rossum', 'centrum', 'wiskunde', 'informatica', 'netherlands', 'successor', 'abc', 'programming', 'language', 'inspired', 'setl', 'capable', 'exception', 'handling', 'interfacing', 'amoeba', 'operating', 'system', '

                                                                 Lemmatization
Lemmatization simplifies text data by reducing words to their base form. This improves analysis consistency.

In [62]:
nlt.download('wordnet')
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [63]:
#print(lemmatized_tokens)

Creating a dictionary and a document-term matrix is necessary before applying the LDA algorithm to identify the topics present in a corpus of documents.                              
This step prepares the textual data for an efficient and accurate analysis and increasing the precision of the LDA topic modeling algorithm.

In [64]:
from gensim import corpora

# Create the dictionary from the tokens
dictionary = corpora.Dictionary([lemmatized_tokens])

# Create the document-term matrix from the dictionary and the lemmatized tokens
doc_term_matrix = [dictionary.doc2bow(doc) for doc in [lemmatized_tokens]]


In [65]:
#doc_term_matrix[:]

# Model building

# <span style="color:brown;font-size:20px">Training the LDA model :</span>
In this section, we will use the Latent Dirichlet Allocation (LDA) algorithmto extract topics from our preprocessed documents using gensim. After defining the number of topics, we'll train the model with LdaModel and display the most relevant topics and words for each topic.

In [66]:
# Number of topics to extract
num_topics =4

# Train the LDA model
lda_model = LdaModel(corpus=doc_term_matrix, id2word=dictionary, num_topics=num_topics)

# Display the most relevant topics and words for each topic
for topic_id, topic in lda_model.print_topics(num_topics=num_topics):
    print(f"Topic {topic_id}: {topic}\n")


Topic 0: 0.052*"python" + 0.019*"language" + 0.009*"programming" + 0.008*"used" + 0.007*"statement" + 0.007*"code" + 0.007*"expression" + 0.005*"version" + 0.005*"c" + 0.005*"operator"

Topic 1: 0.053*"python" + 0.014*"language" + 0.009*"programming" + 0.009*"used" + 0.008*"statement" + 0.008*"expression" + 0.007*"code" + 0.006*"us" + 0.006*"use" + 0.006*"library"

Topic 2: 0.032*"python" + 0.009*"language" + 0.006*"statement" + 0.005*"programming" + 0.005*"use" + 0.005*"program" + 0.004*"code" + 0.004*"expression" + 0.004*"library" + 0.004*"used"

Topic 3: 0.026*"python" + 0.011*"language" + 0.008*"statement" + 0.007*"used" + 0.005*"code" + 0.005*"programming" + 0.005*"expression" + 0.005*"library" + 0.005*"also" + 0.005*"operator"



In [67]:
# Display the most relevant topics and words for each topic
for topic_id, topic in lda_model.show_topics(num_topics=num_topics, formatted=False, num_words=10):
    print(f"Topic {topic_id}:")
    for word, _ in topic:
        print(f"- {word}")
    print()

Topic 0:
- python
- language
- programming
- used
- statement
- code
- expression
- version
- c
- operator

Topic 1:
- python
- language
- programming
- used
- statement
- expression
- code
- us
- use
- library

Topic 2:
- python
- language
- statement
- programming
- use
- program
- code
- expression
- library
- used

Topic 3:
- python
- language
- statement
- used
- code
- programming
- expression
- library
- also
- operator



# visualisation

pyLDAVis is the most commonly used and a nice way to visualise the information contained in a topic model. Below is the implementation for LdaModel()

In [69]:
#!pip install pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

C:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.004646  0.001762       1        1  53.751386
0      0.004788 -0.001769       2        1  46.230968
3     -0.004397  0.000501       3        1   0.008835
2     -0.005036 -0.000494       4        1   0.008811, topic_info=             Term        Freq       Total Category  logprob  loglift
861        python  155.000000  155.000000  Default  30.0000  30.0000
587      language   48.000000   48.000000  Default  29.0000  29.0000
1035    statement   22.000000   22.000000  Default  28.0000  28.0000
1151         used   24.000000   24.000000  Default  27.0000  27.0000
837   programming   27.000000   27.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
615          list    0.000674    9.823583   Topic4  -5.9551  -0.2496
121             c    0.000752   13.278248   Topic4  -5.8464  -0.4423
101         block    0.000694   11.084606   Topic4  -5.9265  -0.3419
903       release    0.000706   11.786624   Topic4  -5.9091  -0.3858
640          many    0.000679   11.415610   Topic4  -5.9488  -0.3935

[312 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
24        1  0.490575    allow
24        2  0.490575    allow
27        1  0.560772   allows
27        2  0.560772   allows
30        1  0.553579     also
...     ...       ...      ...
1180      2  0.568098      web
1191      1  0.819966   window
1191      2  0.819966   window
1203      1  0.560241  written
1203      2  0.466867  written

[355 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3])